In [2]:
library(jsonlite)
library(rlist)
library(data.table)
suppressPackageStartupMessages(library("dplyr"))
library(XML)
library(methods)

In [3]:
# We select the xmls with the datas we need
setwd('/home/jovyan/xml/')


In [4]:
suppressWarnings({
    options(stringsAsFactors = FALSE)
# Open different xml in the same time
# We select the xmls with the datas we need
xmlfiles=list.files(pattern='product3')
# We remove the datas about genetic disorders, they have no interest here
xmlfiles=xmlfiles[!xmlfiles=='en_product3_156.xml']
# We put the 35 xml files from  orphadata in data frames
indice_list=1
xmldoc=list()
files=list()
for (i in 1:length(xmlfiles)) 
{
  xmldoc[[indice_list]]=xmlParse(xmlfiles[i]) # we parse the files
  files[[indice_list]]=xmlToDataFrame(nodes = getNodeSet(xmldoc[[i]], "//Disorder")) # we put files into a list
  indice_list=indice_list+1
}
    })

In [5]:
# We remove the disorders that are duplicated in the same xml
indice_list=1
file=list()
for (i in 1:length(files))
    if (length(files[[i]]) > 0)
{
  file[[indice_list]]=files[[i]][!duplicated(files[[i]]), ]
  indice_list=indice_list+1
}

# We get an unique frame with the datas
file=rbindlist(file, use.names = TRUE)

In [6]:
# We only keep the disorders 
file=filter(file, DisorderType=='Malformation syndrome' | DisorderType=='Disease' |
DisorderType=='Morphological anomaly'| DisorderType=='Clinical syndrome' |
DisorderType=='Particular clinical situation in a disease or syndrome' | DisorderType=='Biological anomaly')

In [7]:
# We count how much times an OrphaCode is duplicated
table_count=as.data.frame(table(file$OrphaCode))
names(table_count)=c('OrphaCode','Number of classifications')

In [8]:
# We look for ORPHAcodes which are duplicated at least once
indice_list=1
duplicates=list()
for (i in 1:nrow(table_count))
  if (table_count$`Number of classifications`[i] > 1)
  {
    duplicates[[indice_list]]=table_count[i, ]
    indice_list=indice_list+1
  }
duplicates=rbindlist(duplicates,use.names = TRUE)

In [9]:
# Number of disorders multiclassified
number_of_disorders_multiclassified=nrow(duplicates)
number_of_disorders_multiclassified

[1] 4186

In [10]:
# Percentage of disorders multiclassified
percentage_of_number_of_disorders_multiclassified=number_of_disorders_multiclassified/nrow(table_count)*100
percentage_of_number_of_disorders_multiclassified

[1] 67.32068

In [11]:
# Number of disorders uniclassified
number_of_disorders_uniclassified=nrow(table_count) - number_of_disorders_multiclassified
number_of_disorders_uniclassified

[1] 2032

In [12]:
# Percentage of disorders uniclassified
percentage_of_number_of_disorders_uniclassified=number_of_disorders_uniclassified/nrow(table_count)*100
percentage_of_number_of_disorders_uniclassified

[1] 32.67932

In [13]:
### Stats : min/max, mean, median of the number of classifications for the disorders
summary(table_count$`Number of classifications`)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
  1.000   1.000   2.000   2.303   3.000  13.000 